In [ ]:
import qutip

In [ ]:
#example code

#generate a spin ket
N = 3; J = N/2; m = -J
print("Total States: ", 2*J+1)
ket = qutip.spin_state(J,m)
print(ket)


# Basis States

This problem is difficult to implement in QuTip without running into exponential scaling.

For this first iteration, we're going to run into it. As it's the easiest code to write.

Our basis states for the atoms are $\{\left|u\right>, \left|d\right>, \left|e\right>\}^{N}$.

In [ ]:
u = qutip.basis(3,0)
d = qutip.basis(3,1)
e = qutip.basis(3,2)

#print out the basis states
# print("u: ", u)
# print("d: ", d)
# print("e: ", e)

id_atom = qutip.qeye(3)

Our cavity fock states are $\{0, 1, 2\}$

In [ ]:
zero = qutip.basis(3,0)
one = qutip.basis(3,1)
two = qutip.basis(3,2)

id_cav = qutip.qeye(3)

Here we define the single-spin operators `sx`, `sy`, `sz` in terms of $\{\left|u\right>, \left|d\right>\}$.

In [ ]:
#create the sx, sy, sz operators on the spin out of the u, d basis

sz = (u*u.dag() - d*d.dag())/2
sx = (u*d.dag() + d*u.dag())/2
sy = (u*d.dag() - d*u.dag())/(2*1j)

#print out the operators
# print("sz: ", sz)
# print("sx: ", sx)
# print("sy: ", sy)

#multiply by the identity on the cavity
sz = qutip.tensor(sz, id_cav)
sx = qutip.tensor(sx, id_cav)
sy = qutip.tensor(sy, id_cav)

Here we define the single-excitation operators for down and up, `nu` and `mu`.

$\nu^\dag = \left| e \right>\left< u \right|$

$\mu^\dag = \left| e \right>\left< d \right|$

In [ ]:
nu = u * e.dag()
mu = d * e.dag()

#multiply by the identity on the cavity
mu = qutip.tensor(mu, id_cav)
nu = qutip.tensor(nu, id_cav)

Here we define the annihilation operator.

In [ ]:
a = qutip.destroy(3)

#multiply by the identity on the spin
a = qutip.tensor(id_atom, a)

Here we define the constants. We'll write everything in terms of 1 scale, the time constant of one of the matrix coefficients.

In [ ]:
import numpy as np

Gamma_si = 180e3/2/np.pi
kappa_si = 500e3/2/np.pi
eta = 4
g_si = 2*np.pi*eta*np.sqrt(Gamma_si*kappa_si)/4

#print a coupling constant in engineering notation
print(f"Atom Coupling Constant: {g_si:.2e} rad/s")

In [ ]:
omega = 2*np.pi*5e6

print(f"Optical Rabi Frequency: {omega:.2e} rad/s")

Here we define the rescaled coefficients.

In [ ]:
scale = g_si

g = g_si/scale
kappa = kappa_si/scale
Gamma = Gamma_si/scale
omega = omega/scale
Lambda = 0.1*kappa/scale

Here we define the Hamiltonian

In [ ]:
def H(t,args):
    try:
        w = args['w']
    except:
        w = 0
    Hpart = g*(a*nu.dag()*np.cos(omega*t)- 1j*a*mu.dag()*np.sin(omega*t)) + Lambda*(np.exp(1j*w*t)*a)

    Ht = Hpart + Hpart.dag()
    return Ht

# Initial State 

In [ ]:
psi0 = qutip.tensor(d, zero)
rho0 = psi0*psi0.dag()
rho0

In [ ]:
id_spin = u*u.dag() + d*d.dag()

id_spin = qutip.tensor(id_spin, id_cav)
id_spin

In [ ]:
cavity_decay = a*np.sqrt(kappa)
atom_decay = nu*np.sqrt(Gamma)
atom_decay

In [ ]:
from qutip import mesolve
from qutip.ui.progressbar import BaseProgressBar, TextProgressBar

print(rho0.dims)
print(cavity_decay.dims)
print(atom_decay.dims)
print(sx.dims)
print(sy.dims)
print(sz.dims)

solution = mesolve(
    H=H,
    rho0=rho0,
    tlist=np.linspace(0, 30/kappa, 100),
    c_ops=[cavity_decay],
    e_ops=[sx*sx, sy*sy, sz*sz, id_spin, a.dag()*a],
    args={'w': -3*kappa},
    progress_bar=TextProgressBar()
)

In [ ]:
#plot sx*sx

import matplotlib.pyplot as plt

sx = solution.expect[0]
sy = solution.expect[1]
sz = solution.expect[2]
norm = solution.expect[3]
photon = solution.expect[4]

#elementwise division
sx = sx/norm
sy = sy/norm
sz = sz/norm



In [ ]:
#plot sx*sx
plt.plot(solution.times, photon, label="sy")